### Importar librerias

In [1]:
import pandas as pd
import queue
import numpy as np

### Leer dataset

In [2]:
data = pd.read_csv("Titanic.csv")
#data

In [3]:
datos = [['Rainy','Hot','High','False','No'], ['Rainy', 'Hot','High','True','No'], ['Overcast', 'Hot','High','False','Yes'],
            ['Sunny', 'Mild','High','False','Yes'], ['Sunny', 'Cool', 'Normal', 'False', 'Yes'],
               ['Sunny', 'Cool', 'Normal', 'True', 'No'], ['Overcast', 'Cool', 'Normal', 'True', 'Yes'],
                 ['Rainy', 'Mild', 'High', 'False', 'No'], ['Rainy', 'Cool', 'Normal', 'False', 'Yes'],
                     ['Sunny', 'Mild', 'Normal', 'False', 'Yes'], ['Rainy', 'Mild', 'Normal', 'True', 'Yes'],
                         ['Overcast', 'Mild', 'High', 'True', 'Yes'], ['Overcast', 'Hot', 'Normal', 'False', 'Yes'],
                             ['Sunny', 'Mild', 'High', 'True', 'No']]

play_golf = pd.DataFrame(datos, columns = ['Outlook', 'Temp', 'Humidity', 'Windy', 'Play_Golf'])
play_golf


,Outlook,Temp,Humidity,Windy,Play_Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


### Estructura del árbol

In [63]:
class Arbol:
    def __init__(self, raiz):
        self.hijos = []
        self.atributos = []
        self.raiz = raiz

    def agregarElemento(self, nodo, nodoPadre, atributo = ""):
        subarbol = self.buscarSubarbol(nodoPadre)
        subarbol.hijos.append(Arbol(nodo))
        subarbol.atributos.append(atributo)
    
    
    def buscarSubarbol(self, nodo):
        if self.raiz == nodo:
            return self
        for subarbol in self.hijos[::-1]:
            arbolBuscado = subarbol.buscarSubarbol(nodo)
            if (arbolBuscado != None):
                return arbolBuscado
        return None  

    
    def profundidad(self):
        if len(self.hijos) == 0: 
            return 1
        else:
            maxprof = 0;
            for subarbol in self.hijos:
                prof = subarbol.profundidad()
                if prof > maxprof:
                    maxprof = prof
            return maxprof + 1
    
    
    def grado(self, num):
        num += 1
        for hijo in self.hijos:
            grado(hijo, num)
    
    
    def printArbolProfundidad(self, atributo = "", level = 0):
        print("|" + "-" * level * 20 + " " + atributo + " --> " + self.raiz)
        for hijo in self.hijos:
            hijo.printArbolProfundidad(self.atributos[self.hijos.index(hijo)], level+1)
    
    
    def printArbolAnchura(self, cola = queue.Queue()):
        print(self.raiz)
        if(len(self.hijos) > 0):
            for hijo in self.hijos:
                cola.put(hijo)
        if(cola.qsize() != 0):
            cola.get().printArbolAnchura(cola)

In [58]:
abuela = "Jacqueline Gurney"
marge = "Marge Bouvier"
patty = "Patty Bouvier"
selma = "Selma Bouvier"
bart = "Bart Simpson"
lisa = "Lisa Simpson"
maggie = "Maggie Simpson"
ling = "Ling Bouvier"

arbol = Arbol(abuela)
arbol.agregarElemento(patty, abuela)
arbol.agregarElemento(selma, abuela)
arbol.agregarElemento(ling, selma)
arbol.agregarElemento(marge, abuela)
arbol.agregarElemento(bart, marge)
arbol.agregarElemento(lisa, marge)
arbol.agregarElemento(maggie, marge)

print(arbol.profundidad())
arbol.printArbolProfundidad()
arbol.printArbolAnchura()


3
| Jacqueline Gurney -->
|-------------------- Patty Bouvier -->
|-------------------- Selma Bouvier -->
|---------------------------------------- Ling Bouvier -->
|-------------------- Marge Bouvier -->
|---------------------------------------- Bart Simpson -->
|---------------------------------------- Lisa Simpson -->
|---------------------------------------- Maggie Simpson -->
Jacqueline Gurney
Patty Bouvier
Selma Bouvier
Marge Bouvier
Ling Bouvier
Bart Simpson
Lisa Simpson
Maggie Simpson


### ID3

In [61]:
def id3(df, columna, arbol = None):
    ncol = len(df.columns)        ## Numero de atributos
    ganancias = []                ## Aqui guardo las ganancias
    
    col_class = df[columna]     ## Esta es la columna a predecir
    etiqueta_class = col_class.value_counts();   ## Numero de elementos de cada tipo en la columna a predecir
    entropy_class = np.array(etiqueta_class) / len(col_class.index)  
    ####### Añadido #######
    entropy_class = entropy_class * np.log2(entropy_class)
    #######################
    entropy_class = -np.sum(entropy_class)  ## entropia de la columna a predecir

    
    for i in range(ncol):   ## Por cada Atributo
        
        col = df.iloc[:,i]  ## Cojo la columna
        if col.name != columna: ## Si no es la que queremos predecir
            semicalculo = []    ## No hagas mucho caso a esto por ahora
            numero_etiquetas = df.iloc[:,i].value_counts()  ## Esto es el numero de elementos de cada tipo en la columna seleccionada
            etiquetas = numero_etiquetas.index ## Esto son los distintos elementos, los nombres
            #class_values = data[columna].dropna().unique() ## Nombre de los elementos de la columna a predecir
            class_values = df[columna].dropna().unique()

            for j in etiquetas:  ## Por cada elemento distinto en la columna
                number = []  ## Variable auxiliar
                for z in class_values:  ## Por cada elemento distinto en la columna a predecir
                    #number.append(len(data[(data[[col.name]] == j) & (data[[columna]] == z) ].index)) ## Los cuento
                    #number.append(len(df[(df[[col.name]] == j) & (df[[columna]] == z) ].index))
                    number.append(sum((df[col.name] == j) & (df[columna] == z)))
                
                number = np.array(number) / numero_etiquetas[j]  
                number_log2 = [0 if x==0 else np.log2(x) for x in number] ##Para evitar que se calcule log2(0) (introduce na's)
                #number = number * np.log2(number)
                number = number * number_log2
                entropia = -np.sum(number)  ## Entropia
                semicalculo.append(numero_etiquetas[j] / len(col_class.index) * entropia) ## Aqui guardo un precalculo de la ganancia
            
            ganancias.append(entropy_class - np.sum(semicalculo))  ## ganancia
            
    return ganancias
   
#print(data)
#id3(data.iloc[1:5], 'Survived')
id3(play_golf, 'Play_Golf')

[0.24674981977443933,
 0.02922256565895487,
 0.15183550136234159,
 0.04812703040826949]

### Entrenamiento

In [92]:
def aprendizaje(df, columna, arbol = None, iteracion = 1, padre = "", atributo = ""):

    if(len(df[columna].value_counts()) > 1 and len(df.columns) > 1): ## Si hay más de una clase a predecir y el df tiene más de una columna
        ganancias = id3(df, columna, arbol)  ## Obtener las ganancias del dataframe pasado
        columna_ganadora = df.columns[np.argmax(ganancias)] ## Seleccionar la columna del df con más importancia

        arbol.agregarElemento(columna_ganadora, padre, atributo) ## Añadir esa columna al arbol

        new_df = df.drop(columna_ganadora, axis = 1) ## Obtener el nuevo df eliminando la columna ganadora

        numero_etiquetas = df.loc[:,columna_ganadora].value_counts() ## Contar las diferentes etiquetas de la columna ganadora

        etiquetas = numero_etiquetas.index ## Obtener una lista con las diferentes etiquetas de la columna ganadora

        for i in etiquetas: ## Iteramos por las etiquetas de la columna ganadora
            indices_delete = (df[columna_ganadora] == i).tolist() ## Obtener los indices de las filas donde se encuentra la etiqueta i
            new_new_df = new_df.iloc[indices_delete] ## Obtener el nuevo df se´gun los índices que se acaban de obtener
            
            aprendizaje(new_new_df, columna, arbol, iteracion+1, padre = columna_ganadora, atributo = i) ## Recursivamente llamar a aprendizaje para construir el arbol completo

    else: ## Si solo queda una clase para predecir o no hay más columnas
        arbol.agregarElemento(df.iloc[0][columna], padre, atributo) ## Agregar la predicción de su rama

decision_tree = Arbol("")
aprendizaje(play_golf, 'Play_Golf', arbol = decision_tree, iteracion = 1)
decision_tree.printArbolProfundidad()

|  --> 
|--------------------  --> Outlook
|---------------------------------------- Sunny --> Windy
|------------------------------------------------------------ False --> Yes
|------------------------------------------------------------ True --> No
|---------------------------------------- Rainy --> Humidity
|------------------------------------------------------------ High --> No
|------------------------------------------------------------ Normal --> Yes
|---------------------------------------- Overcast --> Yes


### Predicción

In [ ]:
def prediction(arbol, datos):
    
    